<a href='https://www.hexnbit.com/'> <img src='https://www.hexnbit.com/wp-content/uploads/2019/09/hexnbit_final_66px.png'/> </a>

In [1]:
!python -m pip install --upgrade pip

In [2]:
!pip3 install opencv-python

In [3]:
import cv2
print(cv2.__version__)

4.6.0


# Task 1:
1. Using OpenCV functions, open the webcam / USB camera device and play a live stream
2. Video stream must stop when "x" key is pressed<br>
**Hint-** Refer Section 41 from the content

In [4]:
cap = cv2.VideoCapture(0)    
while(cap.isOpened()):     
    ret, frame = cap.read()   
    cv2.imshow('frame',frame) 
    if cv2.waitKey(1) & 0xFF == ord("x"): 
        break
cv2.destroyAllWindows()
cap.release()        

# Task 2:
1. Add another window which displays grayscale feed<br>
**Hint-** Refer Section 47 from the content

In [5]:
source = cv2.VideoCapture(0)
 
while True:
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Live", gray)
    key = cv2.waitKey(1)
    if key == ord("x"):
        break

cv2.destroyAllWindows()
source.release()

# Task 3:
1. In the live camera feed, using the Haar Cascade Files, detect following:-
    - Face (Enclosed with Yellow Rectangle)
    - Nose (Enclosed with Blue Rectangle)
    - Eyes (Enclosed with Green Rectangle)
    - Smile (Enclosed with Red Rectangle)<br>
**Hint-** Refer Section 50

In [ ]:
import numpy as np
import time
cv2.__file__

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

face = cv2.VideoCapture(0)

while True:
    ret, img = source.read()
    key = cv2.waitKey(1)
    if key == ord("x"):
        break

faces=face_cascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=5)
print(faces)

x,y,w,h=faces[0]
img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
cv2.imshow('img',img)

roi_color = img[y:y+h, x:x+w]

eyes = eye_cascade.detectMultiScale(roi_color)
print(eyes)

for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,255),2)
cv2.imshow('img',img)


cv2.destroyAllWindows()
source.release()


# Task 4:
1. Based on the Facial Features detected in live feed, try to form a logic which may be used to detect if the person is wearing mask or not<br>
**Hint:** What are the facial features not visible when a person is wearing a mask?

2. If mask is worn write "Wearing Mask" on top of face rectangle using Green Colour
3. Else write "No Mask" on top of face rectangle using Red Colour<br>
**Hint:** Look at the picture attached below / in the references of this download,Consider that only one face is detected in the live feed<br>

The algorithm may not be 100% accurate and will depend on lighting conditions / camera placement etc. but try to calibrate the parameters of "detectMultiScale" function<br>
Press Shift+Tab on the function to check all possible parameters

<img src='mask_detection.jpg'/> </a>

In [ ]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
 
cascPath = os.path.dirname(
    cv2.__file__) + "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
model = load_model("mask_recog1.h5")
 
video_capture = cv2.VideoCapture(0)
while True:
    
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    faces_list=[]
    preds=[]
    for (x, y, w, h) in faces:
        face_frame = frame[y:y+h,x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = img_to_array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame =  preprocess_input(face_frame)
        faces_list.append(face_frame)
        if len(faces_list)>0:
            preds = model.predict(faces_list)
        for pred in preds:
            (mask, withoutMask) = pred
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
 
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()